Aggregate the data into one Data Frame using Pandas.  
Pay attention that files may have different names for the same column. Therefore, make sure that you unify the columns names before concating them.

Standardizing header names.


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import statsmodels.api as sm
from scipy.stats import boxcox
pd.options.display.max_rows = 100
## Install xlrd package to load Excel files
#!conda install openpyxl
#!conda install xlrd


In [3]:
# Define file paths
file1 = 'file1.csv'
file2 = 'file2.csv'
file3 = 'file3.csv'

# Read CSV files
df1 = pd.read_csv(file1)
df2 = pd.read_csv(file2)
df3 = pd.read_csv(file3)

df1 = df1.rename(columns= lambda x: x.lower())
df2 = df2.rename(columns= lambda x: x.lower())
df3 = df3.rename(columns= lambda x: x.lower())

# TO FIND OUT where error in customer lifetime value is I used:
# len(df3[df3['customer lifetime value'].isna()==True])
# df3.info() - the 7k int values of df3 were not integrated in, because df1 and df2 were objects
df1['customer lifetime value'] =  pd.to_numeric(df1['customer lifetime value'], errors='coerce')
df2['customer lifetime value'] =  pd.to_numeric(df2['customer lifetime value'], errors='coerce')

# the function can accept any number of dataframes as input thanks to the asterisk *, 
# it will check if each dataframe has a column named 'st'. If the column is found, it will rename it to 'state' using the rename() method.
# Finally, the function will return all the modified dataframes.



def rename_columns(*dataframes):
    for df in dataframes:
        if 'st' in df.columns:
            df.rename(columns={'st': 'state'}, inplace=True)
    return dataframes


df1, df2, df3 = rename_columns(df1, df2, df3)

cus_df = pd.concat([df1, df2, df3])

cus_df


,customer,state,gender,education,customer lifetime value,income,monthly premium auto,number of open complaints,policy type,vehicle class,total claim amount
0,RB50392,Washington,NaN,Master,NaN,0.0,1000.0,1/0/00,Personal Auto,Four-Door Car,2.704934
1,QZ44356,Arizona,F,Bachelor,NaN,0.0,94.0,1/0/00,Personal Auto,Four-Door Car,1131.464935
2,AI49188,Nevada,F,Bachelor,NaN,48767.0,108.0,1/0/00,Personal Auto,Two-Door Car,566.472247
3,WW63253,California,M,Bachelor,NaN,0.0,106.0,1/0/00,Corporate Auto,SUV,529.881344
4,GA49547,Washington,M,High School or Below,NaN,36357.0,68.0,1/0/00,Personal Auto,Four-Door Car,17.269323
...,...,...,...,...,...,...,...,...,...,...,...
7065,LA72316,California,M,Bachelor,23405.987980,71941.0,73.0,0,Personal Auto,Four-Door Car,198.234764
7066,PK87824,California,F,College,3096.511217,21604.0,79.0,0,Corporate Auto,Four-Door Car,379.200000
7067,TD14365,California,M,Bachelor,8163.890428,0.0,85.0,3,Corporate Auto,Four-Door Car,790.784983
7068,UP19263,California,M,College,7524.442436,21941.0,96.0,0,Personal Auto,Four-Door Car,691.200000


 REMOVE DUPLICATES 

In [4]:

cus_df = cus_df.drop_duplicates()
cus_df = cus_df.reset_index() # rows were deleted and index needs to be reset



Deleting and rearranging columns – delete the column customer as it is only a unique identifier for each row of data

In [5]:
cus_df = cus_df.drop(columns=['customer']) 
cus_df

,index,state,gender,education,customer lifetime value,income,monthly premium auto,number of open complaints,policy type,vehicle class,total claim amount
0,0,Washington,NaN,Master,NaN,0.0,1000.0,1/0/00,Personal Auto,Four-Door Car,2.704934
1,1,Arizona,F,Bachelor,NaN,0.0,94.0,1/0/00,Personal Auto,Four-Door Car,1131.464935
2,2,Nevada,F,Bachelor,NaN,48767.0,108.0,1/0/00,Personal Auto,Two-Door Car,566.472247
3,3,California,M,Bachelor,NaN,0.0,106.0,1/0/00,Corporate Auto,SUV,529.881344
4,4,Washington,M,High School or Below,NaN,36357.0,68.0,1/0/00,Personal Auto,Four-Door Car,17.269323
...,...,...,...,...,...,...,...,...,...,...,...
9130,7065,California,M,Bachelor,23405.987980,71941.0,73.0,0,Personal Auto,Four-Door Car,198.234764
9131,7066,California,F,College,3096.511217,21604.0,79.0,0,Corporate Auto,Four-Door Car,379.200000
9132,7067,California,M,Bachelor,8163.890428,0.0,85.0,3,Corporate Auto,Four-Door Car,790.784983
9133,7068,California,M,College,7524.442436,21941.0,96.0,0,Personal Auto,Four-Door Car,691.200000


Working with data types – Check the data types of all the columns and fix the incorrect ones (for ex. customer lifetime value and number of open complaints ). Hint: remove the percentage from the customer lifetime value and truncate it to an integer value.


In [6]:
# remove the percentage from the customer lifetime value and truncate it to an integer value using to_numeric

cus_df['customer lifetime value'] = cus_df['customer lifetime value'].replace('%', '')
# cus_df['customer lifetime value'] =  pd.to_numeric(cus_df['customer lifetime value'], errors='coerce')  <- this is no longer needed, all values are already integers 
cus_df

,index,state,gender,education,customer lifetime value,income,monthly premium auto,number of open complaints,policy type,vehicle class,total claim amount
0,0,Washington,NaN,Master,NaN,0.0,1000.0,1/0/00,Personal Auto,Four-Door Car,2.704934
1,1,Arizona,F,Bachelor,NaN,0.0,94.0,1/0/00,Personal Auto,Four-Door Car,1131.464935
2,2,Nevada,F,Bachelor,NaN,48767.0,108.0,1/0/00,Personal Auto,Two-Door Car,566.472247
3,3,California,M,Bachelor,NaN,0.0,106.0,1/0/00,Corporate Auto,SUV,529.881344
4,4,Washington,M,High School or Below,NaN,36357.0,68.0,1/0/00,Personal Auto,Four-Door Car,17.269323
...,...,...,...,...,...,...,...,...,...,...,...
9130,7065,California,M,Bachelor,23405.987980,71941.0,73.0,0,Personal Auto,Four-Door Car,198.234764
9131,7066,California,F,College,3096.511217,21604.0,79.0,0,Corporate Auto,Four-Door Car,379.200000
9132,7067,California,M,Bachelor,8163.890428,0.0,85.0,3,Corporate Auto,Four-Door Car,790.784983
9133,7068,California,M,College,7524.442436,21941.0,96.0,0,Personal Auto,Four-Door Car,691.200000


In [7]:
# use value counts to see what in the number of complaints is 
counts = cus_df['number of open complaints'].value_counts()
counts
# after viewing the data I decided to treat the / as a differentiating factor from a previous dataset
# f.ex 1/5/ 00 means that customer has 1 open complaint of one category and 5 of other and decided to sum them up 

0         5629
1/0/00    1623
1          765
2          283
1/1/00     247
3          230
4          119
1/2/00      93
1/3/00      60
5           44
1/4/00      29
1/5/00      12
Name: number of open complaints, dtype: int64

Clean the number of open complaints and extract the middle number which is changing between records.

In [8]:
cus_df['number of open complaints'] = cus_df['number of open complaints'].apply(lambda x: x.split('/')[1].strip() if isinstance(x, str) else x)

# as a function it'd look like this and could be more readable code:
# def extract_complaints(x):
#     if isinstance(x, str):
#         return x.str.split('/').str[1].str.strip()
#     else:
#         return x



Filtering data and Correcting typos – Filter the data in state and gender column to standardize the texts in those columns.

In [9]:
 cus_df['state'].value_counts()

California    3030
Oregon        2601
Arizona       1629
Nevada         882
Washington     768
Cali           120
AZ              74
WA              30
Name: state, dtype: int64

In [10]:
# checking which states need to be corrected
cus_df['state'].unique()

array(['Washington', 'Arizona', 'Nevada', 'California', 'Oregon', 'Cali',
       'AZ', 'WA', nan], dtype=object)

In [11]:
def clean_states(x):
    if x in ['Cali', 'California']:
        return 'California'
    elif x in ['AZ', 'Arizona']:
        return 'Arizona'
    elif x in ['WA', 'Washington']:
        return 'Washington'
    else:
        return x

cus_df['state'] = cus_df['state'].apply(clean_states)
# it didnt work with the map function, I used apply instead which is simmilar


In [12]:
cus_df['state'].value_counts()

California    3150
Oregon        2601
Arizona       1703
Nevada         882
Washington     798
Name: state, dtype: int64

In [13]:
cus_df['gender'].value_counts()

F         4557
M         4368
Male        40
female      30
Femal       17
Name: gender, dtype: int64

In [14]:
# checking which genders need to be corrected

cus_df['gender'].unique()

array([nan, 'F', 'M', 'Femal', 'Male', 'female'], dtype=object)

In [15]:

def clean_gender(x):
    if x in ['M', 'Male']:
        return 'Male'
    elif x in ['F', 'Femal',"female"]:
        return 'Female'
    elif np.nan:  pass
    else:
        return 'U'
    
cus_df['gender'] = list(map(clean_gender, cus_df['gender'])) 

In [16]:
cus_df['gender'].unique()

array([None, 'Female', 'Male'], dtype=object)

In [17]:
cus_df

,index,state,gender,education,customer lifetime value,income,monthly premium auto,number of open complaints,policy type,vehicle class,total claim amount
0,0,Washington,None,Master,NaN,0.0,1000.0,0,Personal Auto,Four-Door Car,2.704934
1,1,Arizona,Female,Bachelor,NaN,0.0,94.0,0,Personal Auto,Four-Door Car,1131.464935
2,2,Nevada,Female,Bachelor,NaN,48767.0,108.0,0,Personal Auto,Two-Door Car,566.472247
3,3,California,Male,Bachelor,NaN,0.0,106.0,0,Corporate Auto,SUV,529.881344
4,4,Washington,Male,High School or Below,NaN,36357.0,68.0,0,Personal Auto,Four-Door Car,17.269323
...,...,...,...,...,...,...,...,...,...,...,...
9130,7065,California,Male,Bachelor,23405.987980,71941.0,73.0,0,Personal Auto,Four-Door Car,198.234764
9131,7066,California,Female,College,3096.511217,21604.0,79.0,0,Corporate Auto,Four-Door Car,379.200000
9132,7067,California,Male,Bachelor,8163.890428,0.0,85.0,3,Corporate Auto,Four-Door Car,790.784983
9133,7068,California,Male,College,7524.442436,21941.0,96.0,0,Personal Auto,Four-Door Car,691.200000


- Replacing null values: Replace missing values with means of the column (for numerical columns). Pay attention that the Income feature for instance has 0s which is equivalent to null values. (We assume here that there is no such income with 0 as it refers to missing values)
Hint: numpy.nan is considered of float64 data type.



In [18]:
# replace NaN with 0 
cus_df['customer lifetime value'] = cus_df['customer lifetime value'].fillna(0)

In [19]:
# store in variable the means of the column customer lifetime value
mean_customer_lifetime_value = np.mean(cus_df['customer lifetime value'])
mean_customer_lifetime_value

cus_df['customer lifetime value'] = cus_df['customer lifetime value'].replace(0, mean_customer_lifetime_value)


In [20]:
# income NaN values to zeroes and calc mean, replace 0 with mean 
# cus_df['income'] = cus_df['income'].fillna(0)
# # need to remove zeroes from calculating mean!!!
# mean_income_value = np.mean(cus_df['income'])
# cus_df['income'] = cus_df['income'].replace(0, mean_income_value)


In [21]:
# # monthly premium auto 
# cus_df['monthly premium auto'] = cus_df['monthly premium auto'].fillna(0)
# mean_monthly_premium_auto = np.mean(cus_df['monthly premium auto'])
# cus_df['monthly premium auto'] = cus_df['monthly premium auto'].replace(0, mean_monthly_premium_auto)

In [22]:
# # monthly premium auto 
# cus_df['total claim amount'] = cus_df['total claim amount'].fillna(0)
# mean_total_claim_amount  = np.mean(cus_df['total claim amount'])
# cus_df['total claim amount'] = cus_df['total claim amount'].replace(0, mean_total_claim_amount)

In [23]:
#number of open complaints
# cus_df['number of open complaints'] = cus_df['number of open complaints'].fillna(0)
# mean_number_of_open_complaints  = np.mean(cus_df['number of open complaints'])
# cus_df['number of open complaints'] = cus_df['number of open complaints'].replace(0, mean_number_of_open_complaints)

In [24]:
# fill missing values with 0 
cus_df['number of open complaints'].fillna(0, inplace=True)
def impute_income(df, col_name):
    # Calculate the mean of the non-zero values
    mean_value = np.nanmean(df[col_name])
    
    # Replace 0 values with the mean
    df[col_name] = np.where(df[col_name] == 0, mean_value, df[col_name])
    
    return df
impute_income(cus_df, 'number of open complaints')
 

TypeError: can only concatenate str (not "int") to str

In [25]:
round(cus_df.isna().sum()/len(cus_df),4)*100 

index                        0.00
state                        0.01
gender                       1.35
education                    0.01
customer lifetime value      0.00
income                       0.01
monthly premium auto         0.01
number of open complaints    0.00
policy type                  0.01
vehicle class                0.01
total claim amount           0.01
dtype: float64

In [ ]:
cus_df

- Bucketing the data - Write a function to replace column "State" to different zones. California as West Region, Oregon as North West, and Washington as East, and Arizona and Nevada as Central



In [ ]:
def states_to_regions(x):
    if x in ['California']:
        return 'West Region'
    elif x in ['Oregon']:
        return 'North West'
    elif x in ['Arizona', 'Nevada']:
        return 'Central'
    elif x in ['Washington']:
        return 'East'
    else:
        return x

cus_df['state'] = cus_df['state'].apply(states_to_regions)

In [ ]:
def rename_columns(cus_df):
    cus_df.rename(columns={'state':'region'}, inplace = True)
    return cus_df
rename_columns(cus_df)


- (Optional) In the column `Vehicle Class`, merge the two categories `Luxury SUV` and `Luxury Car` into one category named `Luxury Vehicle`  

- (Optional) Removing outliers using 1.5*IQR technique for all numerical columns.

- (Optional) Standardizing the data – Use string functions to standardize the text data (lower case)

<b>Important: for Activity 3 and Activity 4 , please use the [file Data_Marketing_Customer_Analysis_Round3.csv](./Data/Data_Marketing_Customer_Analysis_Round3.csv) from the [Data](./Data) folder.</b>